In [1]:
import pandas as pd
import numpy as np

# --- Carga de Datos ---
try:
    print("Cargando archivos...")
    df_test4 = pd.read_csv('test4_complete_with_channel.csv')
    df_prod = pd.read_csv('products_competition_clean_company_channel_fixed.csv')
    print("Archivos cargados.")

    # --- INICIA LA CORRECCIÓN ---
    print("\nIniciando limpieza y estandarización...")

    # 1. Seleccionar y renombrar columnas de df_test4 (el primer archivo)
    df_test4_clean = df_test4[[
        'industry', 'subindustry', 'type_of_product', 
        'generic_product_type', 'price_sar', 'company', 
        'unit_of_measurement', 'total_quantity', 'channel'
    ]].copy()

    # 2. Seleccionar y renombrar columnas de df_prod (el segundo archivo)
    df_prod_clean = df_prod.rename(columns={
        'Industry': 'industry',
        'Sub industry': 'subindustry',
        'Type of product': 'type_of_product',
        'Generic product type': 'generic_product_type',
        'Price per unit SAR': 'price_sar',
        'Company': 'company',
        'Unit of measurement [mL,g,units]': 'unit_of_measurement',
        'Total quantity': 'total_quantity',
        'Channel': 'channel'
    })
    
    # Mantener solo las columnas que necesitamos para la unión
    df_prod_clean = df_prod_clean[df_test4_clean.columns]

    # 3. Limpiar la columna de precios en ambos DataFrames
    # Eliminar 'SAR ' y comas, luego convertir a número
    def clean_price(price_column):
        return pd.to_numeric(price_column.astype(str).str.replace('SAR', '').str.replace(',', '').str.strip(), errors='coerce')

    df_test4_clean['price_sar'] = clean_price(df_test4_clean['price_sar'])
    df_prod_clean['price_sar'] = clean_price(df_prod_clean['price_sar'])

    # 4. Limpiar la columna de unidades en ambos DataFrames
    def clean_units(unit_column):
        return unit_column.str.lower().str.strip()

    df_test4_clean['unit_of_measurement'] = clean_units(df_test4_clean['unit_of_measurement'])
    df_prod_clean['unit_of_measurement'] = clean_units(df_prod_clean['unit_of_measurement'])
    
    print("Limpieza completada.")
    # --- TERMINA LA CORRECCIÓN ---

    # --- Unir ambos datasets ---
    competitor_data = pd.concat([df_test4_clean, df_prod_clean], ignore_index=True)
    
    print(f"\nTotal combinado: {competitor_data.shape[0]} filas, {competitor_data.shape[1]} columnas")

    print("\nUnidades de medida únicas encontradas (después de la limpieza):")
    unique_units = competitor_data['unit_of_measurement'].dropna().unique()
    print(unique_units)
    
    # --- Análisis Exploratorio ---
    print("\n--- Análisis de los Datos de la Competencia (Combinado) ---")
    print("\nColumnas finales del DataFrame:")
    print(competitor_data.columns.tolist())
    print("\nPrimeras 5 filas del archivo combinado:")
    print(competitor_data.head())
    
    print("\nÚltimas 5 filas del archivo combinado:")
    print(competitor_data.tail())

except FileNotFoundError as e:
    print(f"\nError: No se encontró un archivo requerido. Detalles: {e}")
except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")

Cargando archivos...
Archivos cargados.

Iniciando limpieza y estandarización...
Limpieza completada.

Total combinado: 3669 filas, 9 columnas

Unidades de medida únicas encontradas (después de la limpieza):
['ml' 'fl oz' 'l' 'g' 'units']

--- Análisis de los Datos de la Competencia (Combinado) ---

Columnas finales del DataFrame:
['industry', 'subindustry', 'type_of_product', 'generic_product_type', 'price_sar', 'company', 'unit_of_measurement', 'total_quantity', 'channel']

Primeras 5 filas del archivo combinado:
  industry subindustry         type_of_product     generic_product_type  \
0     Home        Home  H1-All purpose cleaner  General purpose cleaner   
1     Home        Home  H1-All purpose cleaner  General purpose cleaner   
2     Home        Home  H1-All purpose cleaner  General purpose cleaner   
3     Home        Home  H1-All purpose cleaner  General purpose cleaner   
4     Home        Home  H1-All purpose cleaner  General purpose cleaner   

   price_sar    company unit

In [2]:
# --- Separación de Datos ---
print("\n--- Separando los datos por tipo de medida ---")
volumetric_units = ['ml', 'l', 'fl oz', 'g']

print(f"Separando productos. Unidades consideradas como volumétricas: {volumetric_units}")

# Creamos una columna temporal en minúsculas para una comparación robusta
competitor_data['unit_of_measurement_lower'] = competitor_data['unit_of_measurement'].str.lower()

# Filtramos los DataFrames
volumetric_df = competitor_data[competitor_data['unit_of_measurement_lower'].isin(volumetric_units)].copy()
unit_df = competitor_data[~competitor_data['unit_of_measurement_lower'].isin(volumetric_units)].copy()

# Eliminamos la columna temporal que ya no necesitamos
volumetric_df.drop(columns=['unit_of_measurement_lower'], inplace=True)
unit_df.drop(columns=['unit_of_measurement_lower'], inplace=True)

print(f"\nResultado: Se encontraron {len(volumetric_df)} productos volumétricos y {len(unit_df)} productos por unidades.")

# --- Guardado de Archivos ---
volumetric_output_path = 'competitor_volumetric_products.csv'
unit_output_path = 'competitor_unit_products.csv'

volumetric_df.to_csv(volumetric_output_path, index=False)
unit_df.to_csv(unit_output_path, index=False)

print(f"\n¡Archivos generados exitosamente en tu carpeta!")
print(f"1. '{volumetric_output_path}'")
print(f"2. '{unit_output_path}'")


--- Separando los datos por tipo de medida ---
Separando productos. Unidades consideradas como volumétricas: ['ml', 'l', 'fl oz', 'g']

Resultado: Se encontraron 3437 productos volumétricos y 232 productos por unidades.

¡Archivos generados exitosamente en tu carpeta!
1. 'competitor_volumetric_products.csv'
2. 'competitor_unit_products.csv'


In [3]:
# --- Ingeniería de Características para Datos Volumétricos (Versión Mejorada) ---

# Cargar el conjunto de datos volumétricos
vol_file_path = 'competitor_volumetric_products.csv'

try:
    vol_df = pd.read_csv(vol_file_path)
    print(f"Archivo '{vol_file_path}' cargado. Contiene {len(vol_df)} filas.")

    # --- INICIA LA CORRECCIÓN ---

    # 1. Limpiar y estandarizar la columna de unidades
    vol_df['unit_of_measurement'] = vol_df['unit_of_measurement'].str.lower().str.strip()

    # 2. Excluir productos medidos en gramos (masa vs. volumen)
    original_rows = len(vol_df)
    vol_df = vol_df[vol_df['unit_of_measurement'] != 'g']
    print(f"\nSe han excluido {original_rows - len(vol_df)} productos medidos en gramos por no ser unidades de volumen.")

    # 3. Crear una función de conversión más clara
    def convert_to_liters(row):
        unit = row['unit_of_measurement']
        quantity = row['total_quantity']
        
        if unit == 'ml':
            return quantity / 1000  # Convertir mililitros a litros
        elif unit == 'fl oz':
            return quantity * 0.0295735  # Convertir onzas líquidas a litros
        elif unit == 'l':
            return quantity  # Ya está en litros
        else:
            return None # Devolver nulo para cualquier otra unidad inesperada

    # 4. Aplicar la función de normalización a Litros
    vol_df['volume_liters'] = vol_df.apply(convert_to_liters, axis=1)

    # --- TERMINA LA CORRECCIÓN ---

    # --- Cálculo del Precio por Litro ---
    # Nos aseguramos de manejar divisiones por cero o por volúmenes nulos.
    vol_df.dropna(subset=['volume_liters'], inplace=True) # Eliminar filas donde no se pudo convertir a litros
    vol_df['price_per_liter'] = vol_df['price_sar'] / vol_df['volume_liters']
    vol_df.replace([np.inf, -np.inf], np.nan, inplace=True) # Reemplazar infinitos por NaN
    vol_df.dropna(subset=['price_per_liter'], inplace=True) # Eliminar filas donde el precio/L no se pudo calcular

    print("\nSe han añadido dos nuevas columnas: 'volume_liters' y 'price_per_liter'.")
    print("Ejemplo de los datos procesados:")
    print(vol_df[['price_sar', 'total_quantity', 'unit_of_measurement', 'volume_liters', 'price_per_liter']].head())

    # Guardamos el archivo procesado, listo para el entrenamiento
    vol_processed_path = 'competitor_volumetric_processed.csv'
    vol_df.to_csv(vol_processed_path, index=False)
    print(f"\nDatos volumétricos procesados y guardados en '{vol_processed_path}'. ¡Listos para entrenar el modelo!")


except FileNotFoundError:
    print(f"\nError: No se encontró el archivo '{vol_file_path}'.")
    print("Asegúrate de haber ejecutado el paso anterior correctamente.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

Archivo 'competitor_volumetric_products.csv' cargado. Contiene 3437 filas.

Se han excluido 27 productos medidos en gramos por no ser unidades de volumen.

Se han añadido dos nuevas columnas: 'volume_liters' y 'price_per_liter'.
Ejemplo de los datos procesados:
   price_sar  total_quantity unit_of_measurement  volume_liters  \
0      18.45           500.0                  ml           0.50   
1      20.95           750.0                  ml           0.75   
2      18.99           500.0                  ml           0.50   
3      15.95           500.0                  ml           0.50   
4      18.00           750.0                  ml           0.75   

   price_per_liter  
0        36.900000  
1        27.933333  
2        37.980000  
3        31.900000  
4        24.000000  

Datos volumétricos procesados y guardados en 'competitor_volumetric_processed.csv'. ¡Listos para entrenar el modelo!


In [4]:
import pandas as pd
import numpy as np

# --- Cargar los datos por unidades que separamos al inicio ---
unit_file_path = 'competitor_unit_products.csv'

try:
    unit_df = pd.read_csv(unit_file_path)
    print(f"Archivo '{unit_file_path}' cargado. Contiene {len(unit_df)} filas.")

    # --- 1. Limpieza de Cantidad ---
    # Nos aseguramos de que la columna 'total_quantity' sea numérica y manejamos posibles errores
    unit_df['total_quantity'] = pd.to_numeric(unit_df['total_quantity'], errors='coerce')
    # Eliminamos filas donde la cantidad no es válida (mayor que cero)
    unit_df.dropna(subset=['total_quantity'], inplace=True)
    unit_df = unit_df[unit_df['total_quantity'] > 0]

    # --- 2. Cálculo del Precio por Pieza/Unidad ---
    # Esta será nuestra variable objetivo para el segundo modelo.
    unit_df['price_per_item'] = unit_df['price_sar'] / unit_df['total_quantity']

    # Reemplazamos infinitos y eliminamos NaNs, igual que antes
    unit_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    unit_df.dropna(subset=['price_per_item'], inplace=True)

    print("\nSe ha añadido la columna 'price_per_item'.")
    print("Ejemplo de los datos ya procesados:")
    print(unit_df[['price_sar', 'total_quantity', 'unit_of_measurement', 'price_per_item']].head())

    # --- 3. Guardado del Archivo Procesado ---
    unit_processed_path = 'competitor_unit_processed.csv'
    unit_df.to_csv(unit_processed_path, index=False)

    print(f"\n¡Proceso completado! Los datos por unidades procesados se han guardado en:")
    print(f"'{unit_processed_path}'")

except FileNotFoundError:
    print(f"\nError: No se encontró el archivo '{unit_file_path}'.")
    print("Asegúrate de que el archivo 'competitor_unit_products.csv' (creado en el Paso 2) esté en tu carpeta.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo 'competitor_unit_products.csv' cargado. Contiene 232 filas.

Se ha añadido la columna 'price_per_item'.
Ejemplo de los datos ya procesados:
   price_sar  total_quantity unit_of_measurement  price_per_item
0      21.45            35.0               units        0.612857
1      26.95            35.0               units        0.770000
2      26.38            96.0               units        0.274792
3      32.55            40.0               units        0.813750
4      95.95           300.0               units        0.319833

¡Proceso completado! Los datos por unidades procesados se han guardado en:
'competitor_unit_processed.csv'


In [5]:
# =============================================================================
# PARTE 1: ENTRENAMIENTO Y EVALUACIÓN DE MODELOS (VERSIÓN CORREGIDA)
# =============================================================================
print("--- Iniciando Parte 1: Entrenamiento y Evaluación de Modelos (Versión Corregida) ---")

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# --- 1. Cargar y preparar el historial de cotizaciones de Wayakit ---
# Este será nuestro "conocimiento interno" para el modelo.
try:
    print("\nCargando historial de cotizaciones de Wayakit...")
    df_quotes_raw = pd.read_csv('Global pricing benchmark_V5_28Oct2024(Quotes history).csv')
    df_wayakit_products = pd.read_csv('wayakit_products_to_predict.csv')

    # Limpieza de nombres de columnas
    df_quotes_raw.columns = df_quotes_raw.columns.str.strip()
    df_wayakit_products.columns = df_wayakit_products.columns.str.strip()

    # Filtrar solo cotizaciones aceptadas
    df_quotes_accepted = df_quotes_raw[df_quotes_raw['Accepted quoted price by client (Yes, No, Pending)'].str.strip() == 'Yes'].copy()

    # Limpiar y convertir la columna de precios
    price_col = 'Quoted price per unit (SAR)'
    df_quotes_accepted[price_col] = pd.to_numeric(df_quotes_accepted[price_col].astype(str).str.replace('SAR', '').str.strip(), errors='coerce')
    df_quotes_accepted.dropna(subset=[price_col], inplace=True)
    
    # Renombrar columnas para facilitar la unión
    # **CORRECCIÓN**: Renombrar solo las columnas necesarias para no crear conflictos
    df_quotes_accepted.rename(columns={
    'WAYAKIT product code': 'Product_ID',
    'Quoted price per unit (SAR)': 'approved_quote_price',
    'Channel (B2B or B2C)': 'channel',
    }, inplace=True)
    
    df_wayakit_products_renamed = df_wayakit_products.rename(columns={
    'Industry': 'industry',
    'SubIndustry': 'subindustry',
    'Generic product type': 'generic_product_type',
    'Type_of_product': 'type_of_product'
    })

    # Añadir 'type_of_product' que no está en el historial de cotizaciones
    # df_wayakit_train_base = pd.merge(
    #     df_quotes_accepted,
    #     df_wayakit_products[['Product_ID', 'Type_of_product']].rename(columns={'Type_of_product': 'type_of_product'}),
    #     on='Product_ID',
    #     how='inner'
    # )
    df_wayakit_train_base = pd.merge(
    df_quotes_accepted,
    df_wayakit_products_renamed, # Usamos la tabla con nombres ya corregidos
    on='Product_ID',
    how='inner'
    )
    df_wayakit_train_base['company'] = 'Wayakit'
    print(f"Se prepararon {len(df_wayakit_train_base)} cotizaciones aceptadas de Wayakit para el entrenamiento.")
    print(df_wayakit_products_renamed.head())
    print(df_wayakit_train_base.head())

except Exception as e:
    print(f"Error al preparar los datos de Wayakit: {e}")
    # Creamos un DataFrame vacío si falla para que el resto del script no se rompa
    df_wayakit_train_base = pd.DataFrame()




--- Iniciando Parte 1: Entrenamiento y Evaluación de Modelos (Versión Corregida) ---



Cargando historial de cotizaciones de Wayakit...
Se prepararon 129 cotizaciones aceptadas de Wayakit para el entrenamiento.
            External ID    Product_ID  \
0  product_template_286  FP-FAI-00100   
1  product_template_287  FP-FAI-00101   
2  product_template_288  FP-FAI-00102   
3  product_template_434  FP-FAI-00103   
4  product_template_289  FP-FAI-00200   

                                        Product_Name       Label_Product_Name  \
0  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
1  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
2  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
3  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
4  WAYAKIT GreenTech Faith wood furniture cleaner...   wood furniture cleaner   

             Presentation  Volume_Liters  Pack_quantity_Units  \
0                  700 mL            0.7                  0.0   
1          

In [6]:
# =============================================================================
# PASO 2: ESTRUCTURAR, COMBINAR Y ENTRENAR
# =============================================================================

# --- 2. Cargar y preparar datos de la competencia ---
try:
    print("\n--- Preparando datos de la competencia ---")
    # Datos volumétricos de la competencia
    df_vol_comp = pd.read_csv('competitor_volumetric_processed.csv')
    Q1_vol = df_vol_comp['price_per_liter'].quantile(0.25)
    Q3_vol = df_vol_comp['price_per_liter'].quantile(0.75)
    IQR_vol = Q3_vol - Q1_vol
    upper_bound_vol = Q3_vol + 1.5 * IQR_vol
    df_vol_comp_cleaned = df_vol_comp[df_vol_comp['price_per_liter'] <= upper_bound_vol].copy()
    df_vol_comp_cleaned['approved_quote_price'] = 0.0 # Clave: Para la competencia, este valor es 0

    # Datos por unidades de la competencia
    df_unit_comp = pd.read_csv('competitor_unit_processed.csv')
    Q1_unit = df_unit_comp['price_per_item'].quantile(0.25)
    Q3_unit = df_unit_comp['price_per_item'].quantile(0.75)
    IQR_unit = Q3_unit - Q1_unit
    upper_bound_unit = Q3_unit + 1.5 * IQR_unit
    df_unit_comp_cleaned = df_unit_comp[df_unit_comp['price_per_item'] <= upper_bound_unit].copy()
    df_unit_comp_cleaned['approved_quote_price'] = 0.0 # Clave: Para la competencia, este valor es 0
    print("Datos de la competencia listos.")

except Exception as e:
    print(f"Error al preparar los datos de la competencia: {e}")


# --- 3. Transformar los datos de Wayakit para que coincidan ---
print("\n--- Transformando datos de Wayakit a la estructura correcta ---")
# **CORRECCIÓN**: Eliminar la columna 'Industry' duplicada (con mayúscula) que viene de las cotizaciones
if 'Industry' in df_wayakit_train_base.columns:
    df_wayakit_train_base.drop(columns=['Industry'], inplace=True)

# COLUMNAS FINALES PARA LA UNIÓN (DEBEN SER IDÉNTICAS)
cols_vol = ['industry', 'subindustry', 'type_of_product', 'generic_product_type', 'price_sar', 'company', 'unit_of_measurement', 'total_quantity', 'channel', 'volume_liters', 'price_per_liter', 'approved_quote_price']
cols_unit = ['industry', 'subindustry', 'type_of_product', 'generic_product_type', 'price_sar', 'company', 'unit_of_measurement', 'total_quantity', 'channel', 'price_per_item', 'approved_quote_price']

# Crear el DataFrame volumétrico de Wayakit
df_wayakit_vol_train = df_wayakit_train_base[df_wayakit_train_base['Volume_Liters'] > 0].copy()
df_wayakit_vol_train['volume_liters'] = df_wayakit_vol_train['Volume (L)']
df_wayakit_vol_train['price_per_liter'] = df_wayakit_vol_train['approved_quote_price'] / df_wayakit_vol_train['volume_liters']
df_wayakit_vol_train['price_sar'] = df_wayakit_vol_train['approved_quote_price']
df_wayakit_vol_train['unit_of_measurement'] = np.where(df_wayakit_vol_train['volume_liters'] < 1, 'ml', 'l')
df_wayakit_vol_train['total_quantity'] = np.where(df_wayakit_vol_train['volume_liters'] < 1, df_wayakit_vol_train['volume_liters'] * 1000, df_wayakit_vol_train['volume_liters'])
df_wayakit_vol_train_final = df_wayakit_vol_train[cols_vol].dropna(subset=['price_per_liter'])

# Crear el DataFrame de unidades de Wayakit
df_wayakit_unit_train = df_wayakit_train_base[df_wayakit_train_base['Pack_quantity_Units'] > 0].copy()
df_wayakit_unit_train['price_per_item'] = df_wayakit_unit_train['approved_quote_price']
df_wayakit_unit_train['price_sar'] = df_wayakit_unit_train['approved_quote_price']
df_wayakit_unit_train['total_quantity'] = pd.to_numeric(df_wayakit_unit_train['Quoted quantity'])
df_wayakit_unit_train['unit_of_measurement'] = 'units'
df_wayakit_unit_train_final = df_wayakit_unit_train[cols_unit].dropna(subset=['price_per_item'])

print("Datos de Wayakit transformados. Así se ve el de cotizaciones volumétricas ahora:")
print(df_wayakit_vol_train_final.head(50))




--- Preparando datos de la competencia ---
Datos de la competencia listos.

--- Transformando datos de Wayakit a la estructura correcta ---
Datos de Wayakit transformados. Así se ve el de cotizaciones volumétricas ahora:
                 industry            subindustry  \
0   Facilities management  Facilities management   
1   Facilities management  Facilities management   
2   Facilities management  Facilities management   
3   Facilities management  Facilities management   
4   Facilities management  Facilities management   
5   Facilities management  Facilities management   
6   Facilities management  Facilities management   
7   Facilities management                   Gyms   
8   Facilities management  Facilities management   
9   Facilities management  Facilities management   
10  Facilities management  Facilities management   
11  Facilities management  Facilities management   
12  Facilities management  Facilities management   
13  Facilities management  Facilities management  

In [7]:
# --- 4. Combinar los datos y definir conjuntos de entrenamiento ---
df_train_vol = pd.concat([df_vol_comp_cleaned[cols_vol], df_wayakit_vol_train_final], ignore_index=True)
df_train_unit = pd.concat([df_unit_comp_cleaned[cols_unit], df_wayakit_unit_train_final], ignore_index=True)

print(f"\nTotal para entrenamiento volumétrico: {len(df_train_vol)} filas.")
print(f"Total para entrenamiento de unidades: {len(df_train_unit)} filas.")
print(df_train_unit.head(50))




Total para entrenamiento volumétrico: 3145 filas.
Total para entrenamiento de unidades: 207 filas.
      industry subindustry                       type_of_product  \
0         Home        Home                 H8-Disinfectant wipes   
1         Home        Home                 H8-Disinfectant wipes   
2         Home        Home                 H8-Disinfectant wipes   
3         Home        Home                 H8-Disinfectant wipes   
4         Home        Home                 H8-Disinfectant wipes   
5         Home        Home                 H8-Disinfectant wipes   
6         Home        Home                 H8-Disinfectant wipes   
7         Home        Home                 H8-Disinfectant wipes   
8         Home        Home                 H8-Disinfectant wipes   
9         Home        Home                 H8-Disinfectant wipes   
10        Home        Home                 H8-Disinfectant wipes   
11        Home        Home                 H8-Disinfectant wipes   
12        Home  

In [8]:
print(df_train_vol.tail(50))

                   industry            subindustry  \
3095  Facilities management  Facilities management   
3096  Facilities management  Facilities management   
3097  Facilities management  Facilities management   
3098  Facilities management  Facilities management   
3099  Facilities management  Facilities management   
3100  Facilities management  Facilities management   
3101  Facilities management  Facilities management   
3102  Facilities management  Facilities management   
3103  Facilities management  Facilities management   
3104  Facilities management  Facilities management   
3105  Facilities management  Facilities management   
3106  Facilities management  Facilities management   
3107  Facilities management  Facilities management   
3108  Facilities management  Facilities management   
3109  Facilities management  Facilities management   
3110  Facilities management  Facilities management   
3111  Facilities management  Facilities management   
3112  Facilities management 

In [9]:

# --- 5. Entrenamiento del Modelo Volumétrico ---
features_vol = ['volume_liters', 'type_of_product', 'subindustry', 'company', 'channel', 'approved_quote_price']
target_vol = 'price_per_liter'

X_vol = df_train_vol[features_vol].fillna('Desconocido')
y_vol = df_train_vol[target_vol]
X_vol_encoded = pd.get_dummies(X_vol, columns=['type_of_product', 'subindustry', 'company', 'channel'], drop_first=True)
model_vol_final = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_leaf=2, oob_score=True)
model_vol_final.fit(X_vol_encoded, y_vol)
print("\nModelo volumétrico entrenado.")
print(f"  - Precisión R^2: {model_vol_final.score(X_vol_encoded, y_vol):.4f}")
print(f"  - Precisión OOB: {model_vol_final.oob_score_:.4f}")

# --- 6. Entrenamiento del Modelo por Unidades ---
features_unit = ['total_quantity', 'type_of_product', 'subindustry', 'company', 'channel', 'approved_quote_price']
target_unit = 'price_per_item'

X_unit = df_train_unit[features_unit].fillna('Desconocido')
y_unit = df_train_unit[target_unit]
X_unit_encoded = pd.get_dummies(X_unit, columns=['type_of_product', 'subindustry', 'company', 'channel'], drop_first=True)
model_unit_final = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_leaf=2, oob_score=True)
model_unit_final.fit(X_unit_encoded, y_unit)
print("\nModelo de unidades entrenado.")
print(f"  - Precisión R^2: {model_unit_final.score(X_unit_encoded, y_unit):.4f}")
print(f"  - Precisión OOB: {model_unit_final.oob_score_:.4f}")


Modelo volumétrico entrenado.
  - Precisión R^2: 0.8174
  - Precisión OOB: 0.6083

Modelo de unidades entrenado.
  - Precisión R^2: 0.8327
  - Precisión OOB: 0.6922


In [10]:
# =============================================================================
# PARTE 2: GENERACIÓN DE REPORTE (VERSIÓN FINAL EXPLICADA)
# =============================================================================
import pandas as pd
import numpy as np

print("\n" + "="*60)
print("--- Iniciando Parte 2: Generación de Reporte con Regla de Ganancia del 30% ---")

try:
    vol_model_columns = X_vol_encoded.columns
    unit_model_columns = X_unit_encoded.columns
    print("\nUtilizando los modelos entrenados en memoria...")

    # --- PASO 1: Cargar los datos fuente ---
    print("\n--- PASO 1: Cargando datos fuente... ---")
    df_quotes_raw = pd.read_csv('Global pricing benchmark_V5_28Oct2024(Quotes history).csv')
    df_wayakit = pd.read_csv('wayakit_products_to_predict.csv')
    print(f"Se cargaron {len(df_wayakit)} productos de Wayakit para la predicción.")
    print("Así se ven las primeras filas del archivo de productos Wayakit:")
    print(df_wayakit.head())

    # --- PASO 2: Preparar una tabla limpia solo con las cotizaciones aceptadas ---
    print("\n\n--- PASO 2: Filtrando solo cotizaciones aceptadas (para unir por nombre)... ---")
    df_quotes_raw.columns = df_quotes_raw.columns.str.strip()
    df_quotes_accepted = df_quotes_raw[df_quotes_raw['Accepted quoted price by client (Yes, No, Pending)'].str.strip() == 'Yes'].copy()
    print(f"Se encontraron {len(df_quotes_accepted)} cotizaciones aceptadas en total (incluyendo varias para el mismo producto).")
    price_col = 'Quoted price per unit (SAR)'
    df_quotes_accepted[price_col] = pd.to_numeric(df_quotes_accepted[price_col].astype(str).str.replace('SAR', '', regex=False).str.strip(), errors='coerce')
    
    quotes_df = df_quotes_accepted[['WAYAKIT Product name', price_col]].rename(columns={
        'WAYAKIT Product name': 'Product_Name',
        'Quoted price per unit (SAR)': 'approved_quote_price'
    }).drop_duplicates(subset='Product_Name', keep='last')
    print(f"De esas, hay {len(quotes_df)} productos ÚNICOS con una cotización aceptada (se usa la más reciente).")
    print("\nTabla creada con los precios de cotizaciones para unir por nombre ('quotes_df'):")
    print(quotes_df.head(10))
    
    # **NUEVO**: Limpiar los nombres de productos en AMBAS tablas antes de unir
    print("\nLimpiando nombres de productos para asegurar que coincidan (quitando espacios y convirtiendo a minúsculas)...")
    quotes_df['Product_Name'] = quotes_df['Product_Name'].str.strip().str.lower()
    df_wayakit['Product_Name'] = df_wayakit['Product_Name'].str.strip().str.lower()
    
    print(quotes_df.head(10))
    print(df_wayakit.head(10))
    

    # --- PASO 3: Unir la lista de productos con sus cotizaciones ---
    print("\n\n--- PASO 3: Uniendo productos y cotizaciones por 'Product_Name'... ---")
    df_wayakit.columns = df_wayakit.columns.str.strip()
    df_wayakit.rename(columns={'SubIndustry': 'subindustry', 'Industry': 'industry', 'Generic product type': 'generic_product_type', 'Type_of_product':'type_of_product'}, inplace=True)
    
    df_wayakit = pd.merge(df_wayakit, quotes_df, on='Product_Name', how='left')
    df_wayakit['approved_quote_price'] = df_wayakit['approved_quote_price'].fillna(0.0)
    print("Resultado de la unión (productos con precio de cotización > 0):")
    print(df_wayakit[df_wayakit['approved_quote_price'] > 0][['Product_Name', 'approved_quote_price']].head(15))
    total_uniones = len(df_wayakit[df_wayakit['approved_quote_price'] > 0])
    print(f"\nEn total, la unión por nombre encontró {total_uniones} coincidencias exitosas en todo el archivo.")
    
    # **DIAGNÓSTICO**: Encontrar y mostrar los nombres que NO coincidieron
    nombres_en_cotizaciones = set(quotes_df['Product_Name'].unique())
    nombres_en_productos = set(df_wayakit['Product_Name'].unique())
    nombres_no_encontrados = nombres_en_cotizaciones - nombres_en_productos

    if nombres_no_encontrados:
        print("\n-----------------------------------------------------------------")
        print("AVISO: Los siguientes productos de tus cotizaciones NO se encontraron en tu lista de productos maestra:")
        for nombre in sorted(list(nombres_no_encontrados)):
            print(f" - {nombre}")
        print("-----------------------------------------------------------------")



    # --- PASO 4: Preparar las características finales para la predicción ---
    print("\n\n--- PASO 4: Creando columnas 'company' y 'channel'... ---")
    df_wayakit['company'] = 'Wayakit'
    b2c_subindustries = ['home', 'automotive', 'pets']
    df_wayakit['channel'] = np.where(df_wayakit['subindustry'].str.lower().isin(b2c_subindustries), 'B2C', 'B2B')
    df_wayakit['Pack_quantity_Units'] = df_wayakit['Pack_quantity_Units'].fillna(0)
    print("Vista final de la tabla antes de entrar al bucle de predicción:")
    print(df_wayakit[['Product_Name', 'company', 'channel', 'approved_quote_price']].head(10))

    # --- PASO 5: Bucle de Predicción y Creación del Reporte ---
    print("\n\n--- PASO 5: Iniciando predicción producto por producto... ---")
    report_list = []
    
    for index, row in df_wayakit.iterrows():
        is_volumetric = row['Volume_Liters'] > 0
        is_unit = row['Pack_quantity_Units'] > 0
        single_product_df = pd.DataFrame([row])
        model_predicted_price = 0
        predicted_price_per_unit = 0

        if is_volumetric:
            single_product_df.rename(columns={'Volume_Liters': 'volume_liters'}, inplace=True)
            features_to_encode = ['volume_liters', 'type_of_product', 'subindustry', 'company', 'channel', 'approved_quote_price']
            X_pred_encoded = pd.get_dummies(single_product_df[features_to_encode], columns=['type_of_product', 'subindustry', 'company', 'channel'])
            X_pred_aligned = X_pred_encoded.reindex(columns=vol_model_columns, fill_value=0)
            predicted_price_per_unit = model_vol_final.predict(X_pred_aligned)[0]
            model_predicted_price = predicted_price_per_unit * row['Volume_Liters']
        elif is_unit:
            single_product_df.rename(columns={'Pack_quantity_Units': 'total_quantity'}, inplace=True)
            features_to_encode = ['total_quantity', 'type_of_product', 'subindustry', 'company', 'channel', 'approved_quote_price']
            X_pred_encoded = pd.get_dummies(single_product_df[features_to_encode], columns=['type_of_product', 'subindustry', 'company', 'channel'])
            X_pred_aligned = X_pred_encoded.reindex(columns=unit_model_columns, fill_value=0)
            predicted_price_per_unit = model_unit_final.predict(X_pred_aligned)[0]
            model_predicted_price = predicted_price_per_unit * row['Pack_quantity_Units']
        
        cost = row['Unit_cost_SAR']
        min_profitable_price = cost * 1.30
        final_price = max(model_predicted_price, min_profitable_price)
        profit_margin = ((final_price - cost) / cost) * 100 if cost > 0 else 0

        report_list.append({
            'Product_ID': row['Product_ID'], 
            'product_name': row['Product_Name'], 
            'product_type': row.get('type_of_product', 'N/A'),
            'generic_product_type': row.get('generic_product_type', 'N/A'),
            'subindustry': row.get('subindustry', 'N/A'), 
            'industry': row.get('industry', 'N/A'),
            'volume': f"{row['Volume_Liters']} L" if is_volumetric else f"{int(row['Pack_quantity_Units'])} units",
            'cost_per_unit': cost, 
            'predicted_price': round(final_price, 2), 
            'predicted_price_per_unit': round(predicted_price_per_unit, 2),
            'porcentaje_de_ganancia': round(profit_margin, 2)
        })

    # --- PASO 6: Guardar y mostrar el reporte final ---
    print("\n\n--- PASO 6: Creando y guardando el reporte final... ---")
    report_df = pd.DataFrame(report_list)
    output_filename = 'wayakit_prediction_report_final_explicado.csv'
    report_df.to_csv(output_filename, index=False)

    print(f"\n¡Proceso de predicción completado!")
    print(f"El reporte detallado se ha guardado en '{output_filename}'")
    
    loss_products_count = (report_df['predicted_price'] <= report_df['cost_per_unit']).sum()
    print("\n" + "-"*50)
    print("Análisis de Rentabilidad:")
    print(f"Se encontraron {loss_products_count} productos cuyo precio de venta predicho es MENOR o IGUAL a su costo.")
    print("-" * 50)

    print("\n--- Vista Previa del Reporte Final ---")
    print(report_df.head().to_string())

except NameError:
    print(f"\nERROR: No se encontraron los modelos. Ejecuta la 'PARTE 1' primero.")
except Exception as e:
    print(f"\nOcurrió un error inesperado en la predicción: {e}")
    


--- Iniciando Parte 2: Generación de Reporte con Regla de Ganancia del 30% ---

Utilizando los modelos entrenados en memoria...

--- PASO 1: Cargando datos fuente... ---
Se cargaron 681 productos de Wayakit para la predicción.
Así se ven las primeras filas del archivo de productos Wayakit:
            External ID    Product_ID  \
0  product_template_286  FP-FAI-00100   
1  product_template_287  FP-FAI-00101   
2  product_template_288  FP-FAI-00102   
3  product_template_434  FP-FAI-00103   
4  product_template_289  FP-FAI-00200   

                                        Product_Name       Label_Product_Name  \
0  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
1  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
2  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
3  WAYAKIT GreenTech Faith general purpose cleane...  general purpose cleaner   
4  WAYAKIT GreenTech Faith wood furniture cleaner...   w